In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
import tensorflow as tf
import time

np.random.seed(42)
tf.random.set_seed(42)

def load_data(file_path):
    df = pd.read_csv(file_path)
    print(f"Loaded data with shape: {df.shape}")
    return df

def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r'<.*?>', '', text)
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        text = text.lower()
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

def preprocess_data(df):
    df['processed_content'] = df['Comment'].apply(preprocess_text)
    return df

def vectorize_text(texts, max_features=5000):
    vectorizer = TfidfVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

def apply_clustering(X, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X)
    return clusters

def map_clusters_to_sentiments(df, clusters):
    cluster_sentiments = {0: 'positive', 1: 'neutral', 2: 'negative'}
    df['cluster'] = clusters
    df['sentiment'] = df['cluster'].map(cluster_sentiments)
    sentiment_map = {'positive': 2, 'neutral': 1, 'negative': 0}
    df['sentiment_label'] = df['sentiment'].map(sentiment_map)
    return df

def evaluate_ml_models(X_train, X_test, y_train, y_test):
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Naive Bayes': MultinomialNB(),
        'SVM': SVC(kernel='linear', random_state=42)
    }
    results = {}
    
    for name, model in models.items():
        start_time = time.time()
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        training_time = time.time() - start_time
        mae = mean_absolute_error(y_test, predictions)
        mse = mean_squared_error(y_test, predictions)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, predictions)
        
        results[name] = {
            'accuracy': accuracy,
            'training_time': training_time,
            'mae': mae,
            'mse': mse,
            'rmse': rmse,
            'r2_score': r2,
            'predictions': predictions,
            'model': model
        }
        print(f"{name} Accuracy: {accuracy:.4f}, MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R² Score: {r2:.4f}")
    return results

def build_lstm_model(texts_train, texts_test, y_train, y_test, max_words=10000, max_len=200):
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(texts_train)
    sequences_train = tokenizer.texts_to_sequences(texts_train)
    sequences_test = tokenizer.texts_to_sequences(texts_test)
    X_train_pad = pad_sequences(sequences_train, maxlen=max_len)
    X_test_pad = pad_sequences(sequences_test, maxlen=max_len)
    
    model = Sequential([
        Embedding(max_words, 128, input_length=max_len),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(32)),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    start_time = time.time()
    history = model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=5, batch_size=32, verbose=1)
    training_time = time.time() - start_time
    loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
    predictions = model.predict(X_test_pad)
    predicted_classes = np.argmax(predictions, axis=1)
    
    mae = mean_absolute_error(y_test, predicted_classes)
    mse = mean_squared_error(y_test, predicted_classes)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predicted_classes)
    
    print(f"LSTM Accuracy: {accuracy:.4f}, MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R² Score: {r2:.4f}")
    
    return {'accuracy': accuracy, 'training_time': training_time, 'mae': mae, 'mse': mse, 'rmse': rmse, 'r2_score': r2, 'history': history, 'model': model}

def main(file_path):
    df = load_data(file_path)
    df = preprocess_data(df)
    X, vectorizer = vectorize_text(df['processed_content'])
    clusters = apply_clustering(X)
    df = map_clusters_to_sentiments(df, clusters)
    X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment_label'], test_size=0.2, random_state=42)
    texts_train, texts_test = train_test_split(df['processed_content'], test_size=0.2, random_state=42)
    ml_results = evaluate_ml_models(X_train, X_test, y_train, y_test)
    lstm_result = build_lstm_model(texts_train, texts_test, y_train, y_test)
    return df, ml_results, lstm_result

if __name__ == "__main__":
    file_path = "cleaned_english_yt_comments.csv"
    df, ml_results, lstm_result = main(file_path)


Loaded data with shape: (9490, 3)
Logistic Regression Accuracy: 0.9658, MAE: 0.0411, MSE: 0.0548, RMSE: 0.2341, R² Score: 0.8913
Random Forest Accuracy: 0.9220, MAE: 0.0843, MSE: 0.0969, RMSE: 0.3114, R² Score: 0.8076
Naive Bayes Accuracy: 0.7071, MAE: 0.3662, MSE: 0.5126, RMSE: 0.7160, R² Score: -0.0174
SVM Accuracy: 0.9600, MAE: 0.0458, MSE: 0.0574, RMSE: 0.2396, R² Score: 0.8860
Epoch 1/5


D:\Documents\Zoom\anconda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


238/238 ━━━━━━━━━━━━━━━━━━━━ 42s 145ms/step - accuracy: 0.7100 - loss: 0.7124 - val_accuracy: 0.8730 - val_loss: 0.3090
Epoch 2/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 36s 151ms/step - accuracy: 0.9268 - loss: 0.2022 - val_accuracy: 0.9131 - val_loss: 0.2550
Epoch 3/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 35s 148ms/step - accuracy: 0.9662 - loss: 0.1020 - val_accuracy: 0.9067 - val_loss: 0.3458
Epoch 4/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 36s 150ms/step - accuracy: 0.9849 - loss: 0.0493 - val_accuracy: 0.8804 - val_loss: 0.4884
Epoch 5/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 36s 149ms/step - accuracy: 0.9867 - loss: 0.0453 - val_accuracy: 0.8994 - val_loss: 0.3782
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step
LSTM Accuracy: 0.8994, MAE: 0.1122, MSE: 0.1354, RMSE: 0.3680, R² Score: 0.7313


In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import xgboost as xgb
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
import tensorflow as tf
import time

np.random.seed(42)
tf.random.set_seed(42)

def load_data(file_path):
    df = pd.read_csv(file_path)
    print(f"Loaded data with shape: {df.shape}")
    return df

def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r'<.*?>', '', text)
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        text = text.lower()
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

def preprocess_data(df):
    df['processed_content'] = df['Comment'].apply(preprocess_text)
    return df

def vectorize_text(texts, max_features=5000):
    vectorizer = TfidfVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

def apply_clustering(X, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    return kmeans.fit_predict(X)

def map_clusters_to_sentiments(df, clusters):
    cluster_sentiments = {0: 'positive', 1: 'neutral', 2: 'negative'}
    df['cluster'] = clusters
    df['sentiment'] = df['cluster'].map(cluster_sentiments)
    sentiment_map = {'positive': 2, 'neutral': 1, 'negative': 0}
    df['sentiment_label'] = df['sentiment'].map(sentiment_map)
    return df

def evaluate_ml_models(X_train, X_test, y_train, y_test):
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Naive Bayes': MultinomialNB(),
        'SVM': SVC(kernel='linear', random_state=42),
        'XGBoost': xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42),
        'AdaBoost': AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
    }
    results = {}
    for name, model in models.items():
        start_time = time.time()
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        training_time = time.time() - start_time
        results[name] = {
            'accuracy': accuracy,
            'training_time': training_time,
            'mae': mean_absolute_error(y_test, predictions),
            'mse': mean_squared_error(y_test, predictions),
            'rmse': np.sqrt(mean_squared_error(y_test, predictions)),
            'r2_score': r2_score(y_test, predictions)
        }
        print(f"{name} Accuracy: {accuracy:.4f}")
    return results

def build_lstm_model(texts_train, texts_test, y_train, y_test, max_words=10000, max_len=200):
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(texts_train)
    X_train_pad = pad_sequences(tokenizer.texts_to_sequences(texts_train), maxlen=max_len)
    X_test_pad = pad_sequences(tokenizer.texts_to_sequences(texts_test), maxlen=max_len)
    
    model = Sequential([
        Embedding(max_words, 128, input_length=max_len),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(32)),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    start_time = time.time()
    history = model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=5, batch_size=32, verbose=1)
    training_time = time.time() - start_time
    accuracy = model.evaluate(X_test_pad, y_test, verbose=0)[1]
    return {'accuracy': accuracy, 'training_time': training_time, 'history': history}

def main(file_path):
    df = load_data(file_path)
    df = preprocess_data(df)
    X, vectorizer = vectorize_text(df['processed_content'])
    df = map_clusters_to_sentiments(df, apply_clustering(X))
    X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment_label'], test_size=0.2, random_state=42)
    texts_train, texts_test = train_test_split(df['processed_content'], test_size=0.2, random_state=42)
    ml_results = evaluate_ml_models(X_train, X_test, y_train, y_test)
    lstm_result = build_lstm_model(texts_train, texts_test, y_train, y_test)
    return df, ml_results, lstm_result

if __name__ == "__main__":
    file_path = "cleaned_english_yt_comments.csv"
    df, ml_results, lstm_result = main(file_path)

Loaded data with shape: (9490, 3)
Logistic Regression Accuracy: 0.9658
Random Forest Accuracy: 0.9220
Naive Bayes Accuracy: 0.7071
SVM Accuracy: 0.9600
XGBoost Accuracy: 0.9457


D:\Documents\Zoom\anconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Accuracy: 0.8857
Epoch 1/5


D:\Documents\Zoom\anconda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


238/238 ━━━━━━━━━━━━━━━━━━━━ 46s 159ms/step - accuracy: 0.7017 - loss: 0.7294 - val_accuracy: 0.8999 - val_loss: 0.2438
Epoch 2/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 40s 169ms/step - accuracy: 0.9275 - loss: 0.2029 - val_accuracy: 0.9125 - val_loss: 0.2451
Epoch 3/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 44s 184ms/step - accuracy: 0.9604 - loss: 0.1160 - val_accuracy: 0.9031 - val_loss: 0.3347
Epoch 4/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 40s 170ms/step - accuracy: 0.9735 - loss: 0.0872 - val_accuracy: 0.9115 - val_loss: 0.3251
Epoch 5/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 36s 150ms/step - accuracy: 0.9749 - loss: 0.0767 - val_accuracy: 0.9131 - val_loss: 0.3886


In [3]:
import pandas as pd
import numpy as np
import re
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import xgboost as xgb
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
import tensorflow as tf

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Function to load data
def load_data(file_path):
    df = pd.read_csv(file_path)
    print(f"Loaded data with shape: {df.shape}")
    return df

# Function to preprocess text
def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Remove URLs
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = re.sub(r'\d+', '', text)  # Remove numbers
        text = text.lower()  # Convert to lowercase
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        return text
    return ""

# Apply preprocessing to dataset
def preprocess_data(df):
    df['processed_content'] = df['Comment'].apply(preprocess_text)
    return df

# Function to vectorize text using TF-IDF
def vectorize_text(texts, max_features=5000):
    vectorizer = TfidfVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

# Apply K-Means clustering
def apply_clustering(X, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    return kmeans.fit_predict(X)

# Mapping clusters to sentiment labels
def map_clusters_to_sentiments(df, clusters):
    cluster_sentiments = {0: 'positive', 1: 'neutral', 2: 'negative'}
    df['cluster'] = clusters
    df['sentiment'] = df['cluster'].map(cluster_sentiments)
    sentiment_map = {'positive': 2, 'neutral': 1, 'negative': 0}
    df['sentiment_label'] = df['sentiment'].map(sentiment_map)
    return df

# Function to calculate MAE, MSE, RMSE, and R² Score
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")
    
    return {"MAE": mae, "MSE": mse, "RMSE": rmse, "R2_Score": r2}

# Train and evaluate multiple ML models
def evaluate_ml_models(X_train, X_test, y_train, y_test):
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Naive Bayes': MultinomialNB(),
        'SVM': SVC(kernel='linear', random_state=42),
        'XGBoost': xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42),
        'AdaBoost': AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
    }
    results = {}
    
    for name, model in models.items():
        start_time = time.time()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        training_time = time.time() - start_time
        metrics = calculate_metrics(y_test, y_pred)
        
        results[name] = {
            'accuracy': accuracy,
            'training_time': training_time,
            **metrics  # Merge calculated metrics
        }
        
        print(f"{name} Accuracy: {accuracy:.4f}")
    
    return results

# Build and train LSTM model
def build_lstm_model(texts_train, texts_test, y_train, y_test, max_words=10000, max_len=200):
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(texts_train)
    X_train_pad = pad_sequences(tokenizer.texts_to_sequences(texts_train), maxlen=max_len)
    X_test_pad = pad_sequences(tokenizer.texts_to_sequences(texts_test), maxlen=max_len)
    
    model = Sequential([
        Embedding(max_words, 128, input_length=max_len),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(32)),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    start_time = time.time()
    history = model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=5, batch_size=32, verbose=1)
    training_time = time.time() - start_time
    accuracy = model.evaluate(X_test_pad, y_test, verbose=0)[1]
    
    return {'accuracy': accuracy, 'training_time': training_time, 'history': history}

# Main function to run all steps
def main(file_path):
    df = load_data(file_path)
    df = preprocess_data(df)
    
    # Vectorization using TF-IDF
    X, vectorizer = vectorize_text(df['processed_content'])
    
    # Apply clustering & sentiment mapping
    df = map_clusters_to_sentiments(df, apply_clustering(X))
    
    # Train/Test Split for ML models
    X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment_label'], test_size=0.2, random_state=42)
    
    # Separate texts for LSTM
    texts_train, texts_test = train_test_split(df['processed_content'], test_size=0.2, random_state=42)
    
    # Evaluate ML models
    ml_results = evaluate_ml_models(X_train, X_test, y_train, y_test)
    
    # Train and evaluate LSTM model
    lstm_result = build_lstm_model(texts_train, texts_test, y_train, y_test)
    
    return df, ml_results, lstm_result

# Run script
if __name__ == "__main__":
    file_path = "cleaned_english_yt_comments.csv"
    df, ml_results, lstm_result = main(file_path)


Loaded data with shape: (9490, 3)
Mean Absolute Error (MAE): 0.0411
Mean Squared Error (MSE): 0.0548
Root Mean Squared Error (RMSE): 0.2341
R² Score: 0.8913
Logistic Regression Accuracy: 0.9658
Mean Absolute Error (MAE): 0.0843
Mean Squared Error (MSE): 0.0969
Root Mean Squared Error (RMSE): 0.3114
R² Score: 0.8076
Random Forest Accuracy: 0.9220
Mean Absolute Error (MAE): 0.3662
Mean Squared Error (MSE): 0.5126
Root Mean Squared Error (RMSE): 0.7160
R² Score: -0.0174
Naive Bayes Accuracy: 0.7071
Mean Absolute Error (MAE): 0.0458
Mean Squared Error (MSE): 0.0574
Root Mean Squared Error (RMSE): 0.2396
R² Score: 0.8860
SVM Accuracy: 0.9600
Mean Absolute Error (MAE): 0.0585
Mean Squared Error (MSE): 0.0669
Root Mean Squared Error (RMSE): 0.2587
R² Score: 0.8672
XGBoost Accuracy: 0.9457


D:\Documents\Zoom\anconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Mean Absolute Error (MAE): 0.1196
Mean Squared Error (MSE): 0.1301
Root Mean Squared Error (RMSE): 0.3607
R² Score: 0.7417
AdaBoost Accuracy: 0.8857
Epoch 1/5


D:\Documents\Zoom\anconda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


238/238 ━━━━━━━━━━━━━━━━━━━━ 44s 149ms/step - accuracy: 0.7181 - loss: 0.7141 - val_accuracy: 0.9152 - val_loss: 0.2251
Epoch 2/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 36s 150ms/step - accuracy: 0.9362 - loss: 0.1795 - val_accuracy: 0.9215 - val_loss: 0.2278
Epoch 3/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 36s 151ms/step - accuracy: 0.9655 - loss: 0.0986 - val_accuracy: 0.9183 - val_loss: 0.2821
Epoch 4/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 36s 150ms/step - accuracy: 0.9823 - loss: 0.0591 - val_accuracy: 0.9204 - val_loss: 0.3227
Epoch 5/5
238/238 ━━━━━━━━━━━━━━━━━━━━ 38s 158ms/step - accuracy: 0.9905 - loss: 0.0356 - val_accuracy: 0.9131 - val_loss: 0.3664
